In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from linearmodels.panel import PanelOLS
import datetime


# file_read(file from executed query)
cond1_treat_0_1 = pd.read_csv("./data/treat_0_1_condition1.csv")
cond1_treat_0_2 = pd.read_csv("./data/treat_0_2_condition1.csv")
cond1_treat_1_1 = pd.read_csv("./data/treat_1_1_condition1.csv")
cond1_treat_1_2 = pd.read_csv("./data/treat_1_2_condition1.csv")


cond2_treat_0_1 = pd.read_csv("./data/treat_0_1_condition2.csv")
cond2_treat_0_2 = pd.read_csv("./data/treat_0_2_condition2.csv")
cond2_treat_1_1 = pd.read_csv("./data/treat_1_1_condition2.csv")
cond2_treat_1_2 = pd.read_csv("./data/treat_1_2_condition2.csv")

cond3_treat_0_1 = pd.read_csv("./data/treat_0_1_condition3.csv")
cond3_treat_0_2 = pd.read_csv("./data/treat_0_2_condition3.csv")
cond3_treat_1_1 = pd.read_csv("./data/treat_1_1_condition3.csv")
cond3_treat_1_2 = pd.read_csv("./data/treat_1_2_condition3.csv")

# file_read(original dataset)
df_topic_day = pd.read_csv("./data/topic_day_panel.csv")



ModuleNotFoundError: No module named 'seaborn'

전처리

In [2]:
#original dataset pp
df_topic_day = df_topic_day[df_topic_day['tag'] != '.net']
df_topic_day = df_topic_day[['date', 'tag', 'q_count', 'treat',	'rel_week']]
df_topic_day.columns = ['cdate', 'sede_tag', 'sede_q_count', 'treat', 'rel_week']

df_topic_day['rel_week'] = pd.Categorical(df_topic_day['rel_week'])
df_topic_day['cdate'] = pd.to_datetime(df_topic_day['cdate'])

df_topic_day  = df_topic_day.sort_values(by = ['sede_tag', 'cdate'], ascending=[True, True])
df_topic_day = df_topic_day.set_index(['sede_tag', 'cdate'])

In [ ]:
df_topic_day

In [4]:
cond1_sede = pd.concat([cond1_treat_0_1
                    , cond1_treat_0_2
                    , cond1_treat_1_1
                    , cond1_treat_1_2], axis = 0)
cond2_sede = pd.concat([  cond2_treat_0_1
                        , cond2_treat_0_2
                        , cond2_treat_1_1
                        , cond2_treat_1_2], axis = 0)

cond3_sede = pd.concat([  cond3_treat_0_1
                        , cond3_treat_0_2
                        , cond3_treat_1_1
                        , cond3_treat_1_2], axis = 0)

In [ ]:
cond1_sede.sort_values(by = ['cdate', 'del_yn'])

In [ ]:
cond2_sede.sort_values(by = ['cdate', 'del_yn'])

In [ ]:
cond3_sede.sort_values(by = ['cdate', 'del_yn'])

In [8]:
cond1_sede = cond1_sede.melt(id_vars = ['cdate', 'del_yn'], value_vars=cond1_sede.columns[1:])
cond2_sede = cond2_sede.melt(id_vars = ['cdate', 'del_yn'], value_vars=cond2_sede.columns[1:])
cond3_sede = cond3_sede.melt(id_vars = ['cdate', 'del_yn'], value_vars=cond3_sede.columns[1:])

In [9]:
cond1_sede['cdate'] = pd.to_datetime(cond1_sede['cdate'])
cond2_sede['cdate'] = pd.to_datetime(cond2_sede['cdate'])
cond3_sede['cdate'] = pd.to_datetime(cond3_sede['cdate'])

In [10]:
cond1_sede.columns = ['cdate', 'del_yn', 'sede_tag', 'sede_q_count']
cond2_sede.columns = ['cdate', 'del_yn', 'sede_tag', 'sede_q_count']
cond3_sede.columns = ['cdate', 'del_yn', 'sede_tag', 'sede_q_count']

In [11]:
cond1_sede.loc[cond1_sede['sede_tag'] =='net', 'sede_tag']='.net'
cond2_sede.loc[cond2_sede['sede_tag'] =='net', 'sede_tag']='.net'
cond3_sede.loc[cond3_sede['sede_tag'] =='net', 'sede_tag']='.net'

In [ ]:
cond1_sede

In [13]:
cond1_sede_y = cond1_sede.loc[cond1_sede['del_yn'] ==1, ['cdate', 'sede_tag', 'sede_q_count']]
cond1_sede_n = cond1_sede.loc[cond1_sede['del_yn'] ==0, ['cdate', 'sede_tag', 'sede_q_count']]

cond2_sede_y = cond2_sede.loc[cond2_sede['del_yn'] ==1, ['cdate', 'sede_tag', 'sede_q_count']]
cond2_sede_n = cond2_sede.loc[cond2_sede['del_yn'] ==0, ['cdate', 'sede_tag', 'sede_q_count']]


cond3_sede_y = cond3_sede.loc[cond3_sede['del_yn'] ==1, ['cdate', 'sede_tag', 'sede_q_count']]
cond3_sede_n = cond3_sede.loc[cond3_sede['del_yn'] ==0, ['cdate', 'sede_tag', 'sede_q_count']]

In [14]:
cond1_sede_yn = cond1_sede.groupby(['cdate', 'sede_tag']).sum()[['sede_q_count']].reset_index()
cond2_sede_yn = cond2_sede.groupby(['cdate', 'sede_tag']).sum()[['sede_q_count']].reset_index()
cond3_sede_yn = cond3_sede.groupby(['cdate', 'sede_tag']).sum()[['sede_q_count']].reset_index()

In [ ]:
df_topic_day

In [16]:
def pp_treat(df):
    # chg data type
    df['cdate'] = pd.to_datetime(df['cdate'])

    # set treat 0/1
    cond1 = df['cdate']>='2021-10-06'
    cond2 = df['cdate']<='2022-03-15'
    cond3 = df['cdate']>='2022-10-05'
    cond4 = df['cdate']<='2023-03-14'

    df.loc[(cond1&cond2), 'treat'] = 0
    df.loc[(cond3&cond4), 'treat'] = 1   
    
    # set rel_days
    df.loc[(cond1&cond2), 'rel_days'] = df.loc[(cond1&cond2), 'cdate'] - datetime.datetime(2021,12,1)
    df.loc[(cond3&cond4), 'rel_days'] = df.loc[(cond3&cond4), 'cdate'] - datetime.datetime(2022,11,30)

    # set rel_weeks
    df['rel_week'] = np.floor(df['rel_days'].dt.days/7)

    #chg data type 
    df[['treat', 'rel_week']] = df[['treat', 'rel_week']].astype(int)
    df['rel_week'] = pd.Categorical(df['rel_week'])
    
    #exception for .net
    df = df[df['sede_tag'] != '.net']

    # sort values
    df  = df.sort_values(by = ['sede_tag', 'cdate'], ascending=[True, True])

    # set index
    df = df.set_index(['sede_tag', 'cdate'])

    return df



In [17]:
cond1_sede_yn = pp_treat(cond1_sede_yn)
cond2_sede_yn = pp_treat(cond2_sede_yn)
cond3_sede_yn = pp_treat(cond3_sede_yn)

cond1_sede_y = pp_treat(cond1_sede_y)
cond1_sede_n = pp_treat(cond1_sede_n)

cond2_sede_y = pp_treat(cond2_sede_y)
cond2_sede_n = pp_treat(cond2_sede_n)

cond3_sede_y = pp_treat(cond3_sede_y)
cond3_sede_n = pp_treat(cond3_sede_n)

In [ ]:
cond3_sede_y

### 모델적용

In [19]:
periods = np.arange(-7, 15, 1)

#### Q1.

##### Target 

In [ ]:
df_topic_day

In [21]:
def run_did(df) :
    # 고정 효과 모델 (tag를 고정 효과로 포함하고, rel_week도 설명 변수로 포함)
    mod_quest_vol_so_dyn = PanelOLS.from_formula('sede_q_count ~ C(rel_week) * treat + EntityEffects',
                                                data=df)
    result_dyn = mod_quest_vol_so_dyn.fit(cov_type='clustered', cluster_entity=True)

    # 동적 효과 계수 추출
    coefs = result_dyn.params
    std_errors = result_dyn.std_errors
    conf_int = result_dyn.conf_int()

    coefs_filtered      = coefs.loc[[f'C(rel_week)[T.{i}]:treat' for i in periods]].values
    std_errors_filtered = std_errors.loc[[f'C(rel_week)[T.{i}]:treat' for i in periods]].values

    # 신뢰구간 계산
    ll_95 = coefs_filtered - 1.965 * std_errors_filtered
    ul_95 = coefs_filtered + 1.965 * std_errors_filtered

    # 데이터프레임 생성 (계수와 신뢰구간)
    df_coefs = pd.DataFrame({
        'period'    : periods,
        'Estimate'  : coefs_filtered,
        'll_95'     : ll_95,
        'ul_95'     : ul_95
    })
    return df_coefs


In [22]:
origin_output = run_did(df_topic_day)

cond1_output   = run_did(cond1_sede_yn)
cond1_y_output = run_did(cond1_sede_y)
cond1_n_output = run_did(cond1_sede_n)

cond2_output   = run_did(cond2_sede_yn)
cond2_y_output = run_did(cond2_sede_y)
cond2_n_output = run_did(cond2_sede_n)

cond3_output   = run_did(cond3_sede_yn)
cond3_y_output = run_did(cond3_sede_y)
cond3_n_output = run_did(cond3_sede_n)

In [ ]:
### create a figure with multiple subplots
sharey = True ## 또는 sharey=False
sharex = True ## 또는 sharex=False

fig, axs = plt.subplots(1, 4, figsize = (20, 5), sharey=sharey, sharex=sharex)


axs[0].fill_between(origin_output['period'], origin_output['ll_95'], origin_output['ul_95'], color='gray', alpha=0.5)
axs[0].plot(origin_output['period'], origin_output['Estimate'], color='black', linewidth=2)
axs[0].axvline(x=0,     color='red',    linestyle='-.', linewidth=1)
axs[0].axhline(y=0,     color='black',  linestyle='--')
axs[0].axhline(y=-20.9, color='green',  linestyle='--')
axs[0].set_xlabel("Week Relative to ChatGPT Release", fontsize = 12)
axs[0].set_ylabel("Content Volume Effect and 95% CI", fontsize = 12)
axs[0].set_title("A. Original trend", fontsize=18.5)
axs[0].tick_params(axis='x', rotation=45)
axs[0].grid(False)

axs[1].fill_between(cond2_n_output['period'], cond2_n_output['ll_95'], cond2_n_output['ul_95'], color='gray', alpha=0.5)
axs[1].plot(cond2_n_output['period'], cond2_n_output['Estimate'], color='black', linewidth=2)
axs[1].axvline(x=0,     color='red',    linestyle='-.', linewidth=1)
axs[1].axhline(y=0,     color='black',  linestyle='--')
# axs[1].axvline(x=3, color='blue', linestyle='-.', linewidth=1)
axs[1].axhline(y=-20.9, color='green',  linestyle='--')
axs[1].set_xlabel("Week Relative to ChatGPT Release", fontsize = 12)
axs[1].set_ylabel("Content Volume Effect and 95% CI", fontsize = 12)
axs[1].set_title("B. Snapshot (2023-12-03)", fontsize=18.5)
axs[1].tick_params(axis='x', rotation=45)
axs[1].grid(False)

axs[2].fill_between(cond1_n_output['period'], cond1_n_output['ll_95'], cond1_n_output['ul_95'], color='gray', alpha=0.5)
axs[2].plot(cond1_n_output['period'], cond1_n_output['Estimate'], color='black', linewidth=2)
axs[2].axvline(x=0, color='red', linestyle='-.', linewidth=1)
axs[2].axhline(y=0, color='black', linestyle='--')
# axs[2].axvline(x=3, color='blue', linestyle='-.', linewidth=1)
axs[2].axhline(y=-20.9, color='green',  linestyle='--')
axs[2].set_xlabel("Week Relative to ChatGPT Release", fontsize = 12)
axs[2].set_ylabel("Content Volume Effect and 95% CI", fontsize = 12)
axs[2].set_title("C. Snapshot (2024-10-01)", fontsize=18.5)
axs[2].tick_params(axis='x', rotation=45)
axs[2].grid(False)


axs[3].fill_between(cond3_n_output['period'], cond3_n_output['ll_95'], cond3_n_output['ul_95'], color='gray', alpha=0.5)
axs[3].plot(cond3_n_output['period'], cond3_n_output['Estimate'], color='black', linewidth=2)
axs[3].axvline(x=0, color='red', linestyle='-.', linewidth=1)
axs[3].axhline(y=0, color='black', linestyle='--')
# axs[3].axvline(x=3, color='blue', linestyle='-.', linewidth=1)
axs[3].axhline(y=-20.9, color='green',  linestyle='--')
axs[3].set_xlabel("Week Relative to ChatGPT Release", fontsize = 12)
axs[3].set_ylabel("Content Volume Effect and 95% CI", fontsize = 12)
axs[3].set_title("D.Surviving > 1 year", fontsize=18.5)
axs[3].tick_params(axis='x', rotation=45)
axs[3].grid(False)

fig.tight_layout()

plt.show();

In [ ]:
plt.savefig("Figure2.png", dpi=300, bbox_inches='tight')  # DPI 설정 및 여백 최소화